In [1]:
import tensorflow as tf
import json
from trainer import Trainer
from utils import map_word_to_vector
import numpy as np
import os

In [2]:
with open('102/checkpoints/model_configs.json', 'r') as inp:
    model_configs = json.load(inp)

In [3]:
with open('IntentDetection/word2idx.json', 'r') as inp:
    new_word2idx = json.load(inp)
with open('IntentDetection/char2idx.json', 'r') as inp:
    new_char2idx = json.load(inp)
with open('102/word2idx.json', 'r') as inp:
    old_word2idx = json.load(inp)
with open('102/char2idx.json', 'r') as inp:
    old_char2idx = json.load(inp)

In [4]:
VERSION = 102
params = dict(
    model_configs = model_configs,
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 8192,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

In [5]:
trainer = Trainer(**params)

In [6]:
trainer.build()

INFO:tensorflow:Restoring parameters from 102/checkpoints/train/model.cpkt-1324235


Restoring parameters from 102/checkpoints/train/model.cpkt-1324235


In [7]:
model_configs['vocab_size'] = len(new_word2idx) + 1
model_configs['char_vocab_size'] = len(new_char2idx) + 1

In [8]:
tf.global_variables()

[<tf.Variable 'LanguageModel/decode_W:0' shape=(512, 153360) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/decode_b:0' shape=(153360,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/embedding_weight:0' shape=(1005, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/kernel:0' shape=(1, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/kernel:0' shape=(2, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/kernel:0' shape=(3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/kernel:0' shape=(4, 16, 64) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/bias:0' s

In [9]:
with tf.variable_scope('LanguageModel', reuse=True):
    W_var = tf.get_variable('decode_W')
    b_var = tf.get_variable('decode_b')
    char_var = tf.get_variable('word_embedding/embedding_weight')
    W_ema_var = trainer.ema.average(W_var)
    b_ema_var = trainer.ema.average(b_var)
    char_ema_var = trainer.ema.average(char_var)

In [10]:
W_val = trainer.session.run(W_var)
b_val = trainer.session.run(b_var)
char_val = trainer.session.run(char_var)
W_ema_val = trainer.session.run(W_ema_var)
b_ema_val = trainer.session.run(b_ema_var)
char_ema_val = trainer.session.run(char_ema_var)

In [11]:
new_W_val = map_word_to_vector(new_word2idx, old_word2idx, W_val.T).T
new_b_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_val, axis=-1)), axis=-1)
new_char_val = map_word_to_vector(new_char2idx, old_char2idx, char_val)
new_W_ema_val = map_word_to_vector(new_word2idx, old_word2idx, W_ema_val.T).T
new_b_ema_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_ema_val, axis=-1)), axis=-1)
new_char_ema_val = map_word_to_vector(new_char2idx, old_char2idx, char_ema_val)

In [12]:
assign_W = tf.assign(W_var, new_W_val, validate_shape=False)
assign_b = tf.assign(b_var, new_b_val, validate_shape=False)
assign_char = tf.assign(char_var, new_char_val, validate_shape=False)
assign_W_ema = tf.assign(W_ema_var, new_W_ema_val, validate_shape=False)
assign_b_ema = tf.assign(b_ema_var, new_b_ema_val, validate_shape=False)
assign_char_ema = tf.assign(char_ema_var, new_char_ema_val, validate_shape=False)

In [13]:
trainer.session.run([assign_W, assign_b, assign_char, assign_W_ema, assign_b_ema, assign_char_ema])

[array([[-0.0050141 ,  0.01232511,  0.03269657, ..., -0.00662151,
         -0.00755983, -0.02902828],
        [-0.01014443, -0.06587451, -0.09476849, ...,  0.0359768 ,
          0.03795993, -0.02359007],
        [-0.01018998, -0.00752271,  0.13559338, ...,  0.0057702 ,
          0.01109316,  0.0053838 ],
        ...,
        [-0.00053481, -0.07606182, -0.08742952, ..., -0.01491118,
          0.00509883, -0.00081415],
        [-0.01365545,  0.04918882, -0.03894585, ...,  0.01981383,
          0.02361569,  0.00964692],
        [ 0.00928689,  0.01865495,  0.01663297, ..., -0.01034157,
         -0.02132533,  0.0111831 ]], dtype=float32),
 array([ 3.7358813e-03,  7.2618127e+00,  3.1828282e+00, ...,
        -3.0431986e-02, -2.3002513e-02,  4.0803221e-03], dtype=float32),
 array([[-0.01157946, -0.08745594,  0.07423651, ...,  0.17092058,
         -0.257632  ,  0.3862866 ],
        [ 3.8711333 ,  2.8239796 ,  0.79968923, ..., -1.3197806 ,
          1.1443709 , -1.0142758 ],
        [ 2.977936  

In [14]:
trainer.checkpoint_dir = 'intent_102/checkpoints'
trainer.model_configs = model_configs
trainer.save_configs()

In [15]:
trainer.train_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'train', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'intent_102/checkpoints/train/model.cpkt-1324235'

In [16]:
trainer.test_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'test', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'intent_102/checkpoints/test/model.cpkt-1324235'